In [1]:
#!pip install jira


In [2]:
from jira import JIRA
import pandas

In [15]:
#Variables
jiraUrl = "https://XXXX.atlassian.net/"
jiraUserEmail = "XXX@XXX.XXX"
jiraAPIKey = "XXX" #https://id.atlassian.com/manage-profile/security/api-tokens
jiraUserAssignee = "XXXXXXXXXXXXXXXXXXXXXXXX"
projectName = "DEV"
issueType = "Story"
epicName = "DEV-13"

In [4]:
#Connecting to Jira cloud instance
auth_jira = JIRA(server=jiraUrl)
auth_jira = JIRA(server=jiraUrl,basic_auth=(jiraUserEmail, jiraAPIKey))

In [5]:
#Test connection
issue = auth_jira.issue("DEV-1")

summary = issue.fields.summary         # 'Field level security permissions'
votes = issue.fields.votes.votes       # 440 (at least)

print(summary)

Test issue


# Search a ticket

In [6]:
searchString = "project = DEV AND summary ~ " + "\"" + summary +"\" " + "order by created DESC"
searchString

'project = DEV AND summary ~ "Test issue" order by created DESC'

In [7]:
issues_in_proj = auth_jira.search_issues(searchString)
issues_in_proj

[<JIRA Issue: key='DEV-1', id='10000'>]

# Create a new ticket

In [8]:
def createJiraTicket(ticketDescription, ticketName, issuetype=issueType, Project=projectName):
    new_issue = auth_jira.create_issue(project=Project, summary=ticketName,
                              description=ticketDescription, issuetype={'name': issuetype})
    return new_issue

# Update a ticket

In [9]:
def editTicket(issue, summary, description, priority, jiraUserAssignee):
    issue_dict = {
        'summary': summary,
        'description': description, #'Update description',
        'priority':{'name':priority},
        'assignee': {'accountId':jiraUserAssignee},
    }
    return issue.update(fields=issue_dict)

In [10]:
def editTicketPriority(issue, priority, jiraUserAssignee):
    issue_dict = {
        'priority':{'name':priority},
    }
    return issue.update(fields=issue_dict)

In [11]:
def addIssueToEPIC(issue, epicIDName):
    issue_dict = {
        "parent":{"key":epicIDName}

    }
    return issue.update(fields=issue_dict)

# Add all CSV items as a ticket

In [12]:
#Import CSV into a pandas data frame
df = pandas.read_csv('jira_ticket_data.csv',delimiter=';')

display(df)

,Summary,Description,Priority
0,Test ticket 1,Description 1,Low
1,Test ticket 2,Description 2,Medium
2,Test ticket 3,Description 3,High


In [13]:
print(df.iloc[0]["Summary"])
print(df.iloc[0]["Description"])
print(df.iloc[0]["Priority"])

Test ticket 1
Description 1
Low


In [14]:
#Loop to add all tickets
for item in range(0,len(df)):
    new_issue_created = createJiraTicket(df.iloc[item]["Description"], df.iloc[item]["Summary"])
    print("Ticket creation result: ", new_issue_created)
    resultTicketEditing = editTicketPriority(new_issue_created, df.iloc[item]["Priority"], jiraUserAssignee)
    print("=========================")
    

Ticket creation result:  DEV-21
Ticket creation result:  DEV-22
Ticket creation result:  DEV-23
